In [1]:
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv(r'C:\Users\sumanth\Projects\Product Length Prediction Amazon ML Hackathon\dataset\train.csv')

In [3]:
test=pd.read_csv(r'C:\Users\sumanth\Projects\Product Length Prediction Amazon ML Hackathon\dataset\test.csv')

In [76]:
tr=train
tr.dropna()

PRODUCT_ID                                              TITLE  \
2           2765088  PRIKNIK Horn Red Electric Air Horn Compressor ...   
3           1594019  ALISHAH Women's Cotton Ankle Length Leggings C...   
5           2152929  HINS Metal Bucket Shape Plant Pot for Indoor &...   
7           2026580  Delavala Self Adhesive Kitchen Backsplash Wall...   
9           2998633  Hexwell Essential oil for Home Fragrance Oil A...   
...             ...                                                ...   
2249686     1898841  Cocomii Holographic iPhone 11 Pro Case - Holog...   
2249689     1616561  Have It Tall Men's Curved Hem Long Drop Tail T...   
2249694     2766635  (3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...   
2249695     1987786  Kangroo Sweep Movement Printed Wooden Wall Clo...   
2249697     1072666  Skyjacker C7360SP Component Box For PN[C7360PK...   

                                             BULLET_POINTS  \
2        [Loud Dual Tone Trumpet Horn, Compatible With ...   
3        [Made By 95%cotton and 5% Lycra which gives yo...   
5        [Simple and elegant, great for displaying indo...   
7        [HIGH QUALITY PVC MATERIAL: The kitchen alumin...   
9        [100% Pure And Natural Essential Oil Or Fragra...   
...                                                    ...   
2249686  [COCOMII - UNIQUE, STYLISH, QUALITY: This beau...   
2249689  [Light Weight 65% Polyester/ 35% Ring Spun Cot...   
2249694  [❤ [Inspiration] Inspired by the Untitled Goos...   
2249695  [Dial size: 12 inches in diameter,Big, clear r...   
2249697  [Component Box For PN[C7360PK],4 in. Lift,Incl...   

                                               DESCRIPTION  PRODUCT_TYPE_ID  \
2        Specifications: Color: Red, Material: Aluminiu...             7537   
3        AISHAH Women's Lycra Cotton Ankel Leggings. Br...             2996   
5        HINS Brings you the most Elegant Looking Pot w...             5725   
7        <p><strong>Aluminum Foil Stickers-good kitchen...             6030   
9        Transform your home, workplace or hotel room i...             8201   
...                                                    ...              ...   
2249686  <b>COCOMII</b><br><br>We transform amazing ide...            12556   
2249689  This extra long Tall t-Shirt will be your favo...             2879   
2249694  <p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...             3413   
2249695  Wall Clocks Are Very Attractive In Looks And E...             1574   
2249697  Skyjacker C7360SP Component Box For PN[C7360PK...             7367   

         PRODUCT_LENGTH  
2            748.031495  
3            787.401574  
5            950.000000  
7            984.251967  
9            393.700787  
...                 ...  
2249686      600.000000  
2249689     1200.000000  
2249694      125.984252  
2249695     1200.000000  
2249697     2000.000000  

[1038460 rows x 6 columns]

In [77]:
te=test
te['TITLE'].fillna(method='ffill', inplace=True)  # Forward fill
te['DESCRIPTION'].fillna(method='bfill', inplace=True)  # Backward fill
te['BULLET_POINTS'].fillna('Unknown', inplace=True)  # Fill with a specific value
te['PRODUCT_TYPE_ID'].fillna(te['PRODUCT_TYPE_ID'].median(), inplace=True)

In [78]:
tr['DESCRIPTION'].fillna('', inplace=True)

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [80]:
label_encoder = LabelEncoder()

In [81]:
for col in ['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'PRODUCT_TYPE_ID']:
    label_encoder.fit(pd.concat([tr[col], te[col]]))
    tr[col] = label_encoder.transform(tr[col])
    te[col] = label_encoder.transform(te[col])


In [84]:
X = tr.drop(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'PRODUCT_TYPE_ID'], axis=1)
y=tr['PRODUCT_LENGTH']
X_test = te.drop(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'PRODUCT_TYPE_ID'], axis=1)
#y_test = tr['PRODUCT_LENGTH']

In [85]:
XTrain, X_test, yTrain, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [86]:
model = RandomForestRegressor(n_estimators=100, random_state=42) 
model.fit(XTrain, yTrain)

RandomForestRegressor(random_state=42)

In [87]:
y_pred = model.predict(X_test)
y_pred

array([492.12598424, 500.        ,   2.        , ..., 420.        ,
       590.55118051, 399.99999924])

In [88]:
results_df = pd.DataFrame({'PRODUCT_ID': X_test.index, 'PRODUCT_LENGTH': y_pred})
results_df

PRODUCT_ID  PRODUCT_LENGTH
0            90943      492.125984
1          1001689      500.000000
2           536614        2.000000
3           369746     3150.000000
4           175997      700.000000
...            ...             ...
449935      256513      590.551181
449936     1711718      708.661417
449937     2072617      420.000000
449938     1644801      590.551181
449939     1364355      399.999999

[449940 rows x 2 columns]

In [89]:
y_test

90943       492.125984
1001689     500.000000
536614        2.000000
369746     3150.000000
175997      700.000000
              ...     
256513      590.551180
1711718     708.661417
2072617     420.000000
1644801     590.551180
1364355     400.000000
Name: PRODUCT_LENGTH, Length: 449940, dtype: float64

In [90]:
score = max(0, 100 * (1- metrics.mean_absolute_percentage_error(y_test, y_pred)))


In [91]:
score

99.99939936617885

In [92]:
model.score(XTrain,yTrain)

0.9063882099886411

In [ ]:
results_df.to_csv('results.csv', index=False)